# Portfolio Selection Optimization
This model is an example of the classic [Markowitz portfolio selection optimization model](https://en.wikipedia.org/wiki/Markowitz_model). We want to find the fraction of the portfolio to invest among a set of stocks that balances risk and return. It is a Quadratic Programming (QP) model with vector and matrix data for returns and risk, respectively. This is best suited to a matrix formulation, so we use the Gurobi Python *matrix* interface. The basic model is fairly simple, so we also solve it parametrically to find the efficient frontier.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 


## Model Formulation
### Parameters

We use the [Greek values](https://en.wikipedia.org/wiki/Greeks_\(finance\)) that are traditional in finance:

- $\delta$: n-element vector measuring the change in price for each stock
- $\sigma$: n x n matrix measuring the covariance among stocks

There is one additional parameter when solving the model parametrically:

- r: target return


### Decision Variables
- $x \ge 0$: n-element vector where each element represents the fraction of the porfolio to invest in each stock

### Objective Function
Minimize the total risk, a convex quadratic function:

\begin{equation}
\min x^t \cdot \sigma \cdot x
\end{equation}

### Constraints

Allocate the entire portfolio: the total investments should be 1.0 (100%), where $e$ is a unit vector (all 1's):

\begin{equation}
e \cdot x = 1
\end{equation}


Return: When we solve the model parametrically for different return values $r$, we add a constraint on the target return:

\begin{equation}
\delta \cdot x = r
\end{equation}

## Python Implementation
### Stock data
Use [yfinance](https://pypi.org/project/yfinance/) library to get the latest 2 years of _actual stock data_ from the 20 most profitable US companies, [according to Wikipedia in April 2021](https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue#List_of_companies_by_profit).

In [28]:
%pip install gurobipy yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import yfinance as yf

stocks = ['BBAS3.SA', 'ECOR3.SA', 'ITUB4.SA', 'MGLU3.SA', 'RADL3.SA', 'UNIP5.SA']
start_date = "2017-01-01"
end_date = "2017-12-31"
data = yf.download(stocks, start=start_date, end=end_date, interval='1mo')

[*********************100%***********************]  6 of 6 completed


### Compute Greeks
Using the downloaded stock data, find the delta (return), sigma (covariance) and standard deviation values for stock prices:

In [45]:
import numpy as np

closes = np.transpose(np.array(data.Close)) # matrix of daily closing prices
absdiff = np.diff(closes)                   # change in closing price each day
reldiff = np.divide(absdiff, closes[:,:-1]) # relative change in daily closing price
delta = np.mean(reldiff, axis=1)            # mean price change
sigma = np.cov(reldiff)                     # covariance (standard deviations)
std = np.std(reldiff, axis=1)               # standard deviation

In [46]:
print(delta)

[0.00561259 0.03479671 0.01373673 0.18820082 0.03339158 0.07461802]


## Minimize risk by solving QP model

In [47]:
import gurobipy as gp
from gurobipy import GRB
from math import sqrt

# Create an empty model
m = gp.Model('portfolio')

# Add matrix variable for the stocks
w1 = m.addVar(lb=0)
w2 = m.addVar(lb=0)
w3 = m.addVar(lb=0)
w4 = m.addVar(lb=0)
w5 = m.addVar(lb=0)
w6 = m.addVar(lb=0)

coefficients = [0.0141, 0.020933, 0.03644, 0.03666, 0.073917, 0.190333]
weights = [w1, w4, w2, w6, w3, w5]

Rp = gp.LinExpr()
for i in range(len(coefficients)):
    Rp += coefficients[i] * weights[i]
m.setObjective(Rp, GRB.MAXIMIZE)

#Restrições
m.addConstr(w1>=0)
m.addConstr(w2>=0)
m.addConstr(w3>=0)
m.addConstr(w4>=0)
m.addConstr(w5>=0)
m.addConstr(w6>=0)
m.addConstr(w6>=0)
m.addConstr(w1 + w2 + w3 + w4 + w5 + w6 == 1)

#Replicando o modelo conservador
m.addConstr(w1 >= 0.05)
m.addConstr(w3 >= 0.05)  # Modified from w2 >= 0.05
m.addConstr(w5 >= 0.05)  # Modified from w3 >= 0.05
m.addConstr(w2 >= 0.05)  # Modified from w4 >= 0.05
m.addConstr(w6 >= 0.05)  # Modified from w5 >= 0.05
m.addConstr(w4 >= 0.05)  # Modified from w6 >= 0.05
m.addConstr(w1 + w3 <= 0.2)  # Modified from w1 + w2 <= 0.2
m.addConstr(w5 + w6 >= 0.3)  # Modified from w3 + w4 >= 0.3
m.addConstr(w2 + w4 >= 0.5)  # Modified from w5 + w6 >= 0.5


# Verify model formulation
m.write('portfolio_selection_optimization.lp')

# Optimize model to find the minimum risk portfolio
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 17 rows, 6 columns and 25 nonzeros
Model fingerprint: 0x96eb77ef
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Presolve removed 17 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0295050e-02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.029505000e-02


In [48]:
print(weights)


[<gurobi.Var C0 (value 0.05)>, <gurobi.Var C3 (value 0.05)>, <gurobi.Var C1 (value 0.45)>, <gurobi.Var C5 (value 0.05)>, <gurobi.Var C2 (value 0.05)>, <gurobi.Var C4 (value 0.35)>]


## Display minimum risk portfolio using Pandas

In [13]:
import pandas as pd
minrisk_volatility = sqrt(m.ObjVal)
minrisk_return = delta @ x.X
pd.DataFrame(data=np.append(x.X, [minrisk_volatility, minrisk_return]),
             index=stocks + ['Volatility', 'Expected Return'],
             columns=['Minimum Risk Portfolio'])

NameError: ignored

## Compute the efficient frontier
Solve the QP parametrically to find the lowest risk portfolio for different expected returns.

In [ ]:
# Create an expression representing the expected return for the portfolio
portfolio_return = delta @ x
target = m.addConstr(portfolio_return == minrisk_return, 'target')

# Solve for efficient frontier by varying target return
frontier = np.empty((2,0))
for r in np.linspace(delta.min(), delta.max(), 25):
    target.rhs = r
    m.optimize()
    frontier = np.append(frontier, [[sqrt(m.ObjVal)],[r]], axis=1)

## Plot results
Use the matplot library to plot the optimized solutions, along with the individual stocks:

In [ ]:
import matplotlib.pyplot as plt
#plt.figure(figsize=(10,10))

fig, ax = plt.subplots(figsize=(10,8))

# Plot volatility versus expected return for individual stocks
ax.scatter(x=std, y=delta,
           color='Blue', label='Individual Stocks')
for i, stock in enumerate(stocks):
    ax.annotate(stock, (std[i], delta[i]))

# Plot volatility versus expected return for minimum risk portfolio
ax.scatter(x=minrisk_volatility, y=minrisk_return, color='DarkGreen')
ax.annotate('Minimum\nRisk\nPortfolio', (minrisk_volatility, minrisk_return),
            horizontalalignment='right')

# Plot efficient frontier
ax.plot(frontier[0], frontier[1], label='Efficient Frontier', color='DarkGreen')

# Format and display the final plot
ax.axis([frontier[0].min()*0.7, frontier[0].max()*1.3, delta.min()*1.2, delta.max()*1.2])
ax.set_xlabel('Volatility (standard deviation)')
ax.set_ylabel('Expected Return')
ax.legend()
ax.grid()
plt.show()